## 函数包测试

In [26]:
pwd

'/home/zhwang/Documents/jupyter/0-items/WeylGroupExtension'

In [2]:
cd Documents/jupyter/0-items/WeylGroupExtension/

/home/zhwang/Documents/jupyter/0-items/WeylGroupExtension


In [27]:
load("weyl_group_extension.sage")

In [4]:
def show_tree(tree):
    """打印群树"""
    for i in tree_str:
        for j in i:
            print(j,end="\t")
        print()
    return

### 测试一 反射群

In [37]:
# 初始化
s,n = "A",3
mat = matrix(CoxeterMatrix([s,n]))
print("Coxeter 矩阵",mat,sep="\n")
G = GroupByOrderMatrix(mat)

# 计算结果
tree1 = GroupTreeOfMaxDepth(G)
tree,tree_str,relations = UniversalPropertyOfGroup(G)
print("两种方法是否相等",tree1[:-1] == tree, sep="\n")
print("打印群树")
show_tree(tree)
relations

Coxeter 矩阵
[1 3 2]
[3 1 3]
[2 3 1]
两种方法是否相等
True
打印群树
	
0	1	2	
0-1	0-2	1-0	1-2	2-1	
0-1-0	0-1-2	0-2-1	1-0-2	1-2-1	2-1-0	
0-1-0-2	0-1-2-1	0-2-1-0	1-0-2-1	1-2-1-0	
0-1-0-2-1	0-1-2-1-0	1-0-2-1-0	
0-1-0-2-1-0	


[('0-0', ''),
 ('1-1', ''),
 ('2-0', '0-2'),
 ('2-2', ''),
 ('1-0-1', '0-1-0'),
 ('2-1-2', '1-2-1'),
 ('2-1-0-2', '1-2-1-0')]

### 测试二 无限群

In [38]:
# 初始化
s,n = "A",2
gens = Thetas(s,n)
order_mat = OrderMatrixOfGens(gens)
print("群阶矩阵（泛性质）",order_mat, sep="\n")
G = GroupByOrderMatrix(order_mat)

# 计算结果
k = 8 # 取前 k 层
tree1 = GroupTreeOfMaxDepth(G,k)
tree,tree_str,relations = UniversalPropertyOfGroup(G,k)
print("两种方法是否相等",tree1 == tree, sep="\n")
relations

群阶矩阵（泛性质）
[2 3]
[3 2]
两种方法是否相等
True


[('0-0-0-0', ''),
 ('1-1-1-1', ''),
 ('0-1-0-1-0', '1-1-1'),
 ('1-0-1-0-1', '0-0-0'),
 ('0-0-0-1-1-1', '1-0-1-0'),
 ('1-1-1-0-0-0', '0-1-0-1'),
 ('1-0-1-0-0-1-0-1', '0-0-0-1-1-0-0-0'),
 ('1-1-1-0-0-1-1-1', '0-1-0-1-1-0-1-0')]

### 测试三 Coxeter 群 I 型

In [39]:
# I型 Coxeter 群，仅当全部遍历时，泛性质才完整
# 初始化
mat = matrix([[1,20],[20,1]])
print("Coxeter 矩阵",mat,sep="\n")
G = GroupByOrderMatrix(mat)

# 打印结果
k = 10
tree1 = GroupTreeOfMaxDepth(G,k)
tree,tree_str,relations = UniversalPropertyOfGroup(G,k)
print("两种方法是否相等",tree1 == tree)
relations

Coxeter 矩阵
[ 1 20]
[20  1]
两种方法是否相等 True


[('0-0', ''), ('1-1', '')]

### 测试四 多余泛性质的规律

In [41]:
# 测试 A 型
for n in range(5):
    print("A%d 型 Coxeter 矩阵："%(n+1))
    mat = matrix(CoxeterMatrix(["A",n+1]))
    print(mat)
    G = GroupByOrderMatrix(mat)
    _,_,relations = UniversalPropertyOfGroup(G)
    print("多余内容：",[r for r in relations if len(set(r[0].replace("-","")))>2])
    print()

A1 型 Coxeter 矩阵：
[1]
多余内容： []

A2 型 Coxeter 矩阵：
[1 3]
[3 1]
多余内容： []

A3 型 Coxeter 矩阵：
[1 3 2]
[3 1 3]
[2 3 1]
多余内容： [('2-1-0-2', '1-2-1-0')]

A4 型 Coxeter 矩阵：
[1 3 2 2]
[3 1 3 2]
[2 3 1 3]
[2 2 3 1]
多余内容： [('2-1-0-2', '1-2-1-0'), ('3-2-1-3', '2-3-2-1'), ('3-2-1-0-3', '2-3-2-1-0')]

A5 型 Coxeter 矩阵：
[1 3 2 2 2]
[3 1 3 2 2]
[2 3 1 3 2]
[2 2 3 1 3]
[2 2 2 3 1]
多余内容： [('2-1-0-2', '1-2-1-0'), ('3-2-1-3', '2-3-2-1'), ('4-3-2-4', '3-4-3-2'), ('3-2-1-0-3', '2-3-2-1-0'), ('4-3-2-1-4', '3-4-3-2-1'), ('4-3-2-1-0-4', '3-4-3-2-1-0')]



### 测试五 “局部泛性质”

In [45]:
def relations2elements(relations,gens):
    """将生成元转为群元素"""
    one = gens[0]/gens[0]
    def str2element(s):
        if not s:return one
        new = one
        for i in s.split("-"):
            new *= gens[int(i)]
        return new
    elements = [str2element(a)/str2element(b) for a,b in relations]
    return elements

In [46]:
# 用前 k 阶泛性质生成商群 -->> 前 k 层元素“相同” （至少）
# 实际计算时，前 k 层的泛性质，可能已经是完整的泛性质了

# 初始化
s,n = "A",2
gens = Thetas(s,n)
order_mat = OrderMatrixOfGens(gens)
print("泛性质矩阵",order_mat,sep="\n")
G = GroupByOrderMatrix(order_mat)

# 计算结果
k = 8
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%k,relations, sep="\n")
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements
[len(i) for i in GroupTreeOfMaxDepth(newG,k)]==[len(i) for i in GroupTreeOfMaxDepth(G,k)]

泛性质矩阵
[2 3]
[3 2]
前8层导出关系
[('0-0-0-0', ''), ('1-1-1-1', ''), ('0-1-0-1-0', '1-1-1'), ('1-0-1-0-1', '0-0-0'), ('0-0-0-1-1-1', '1-0-1-0'), ('1-1-1-0-0-0', '0-1-0-1'), ('1-0-1-0-0-1-0-1', '0-0-0-1-1-0-0-0'), ('1-1-1-0-0-1-1-1', '0-1-0-1-1-0-1-0')]
商集
[x0^4, x1^4, (x0*x1)^2*x0*x1^-3, (x1*x0)^2*x1*x0^-3, x0^3*x1^3*(x0^-1*x1^-1)^2, x1^3*x0^3*(x1^-1*x0^-1)^2, x1*(x0*x1*x0)^2*x1*x0^-3*x1^-2*x0^-3, x1^3*x0^2*x1^3*x0^-1*(x1^-1*x0^-1*x1^-1)^2*x0^-1]


True

In [ ]:
# 实际上，检验前 13 层仍然正确
[len(i) for i in GroupTreeOfMaxDepth(newG,13)]==[len(i) for i in GroupTreeOfMaxDepth(G,13)]

### 测试六 weyl 群的极小泛性质

#### ADE型-取前 3 层

In [ ]:
# 初始化
s,n = "E",6
mat = matrix(CoxeterMatrix([s,n]))
print("Coxeter 矩阵",mat,sep="\n")
G = GroupByOrderMatrix(mat)

# 计算结果
k = 3
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%k,relations, sep="\n")
print()
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements

# 打印结果
n1 = [len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]
n2 = [len(i) for i in GroupTreeOfMaxDepth(G,k+1)]
print("前%d层元素个数是否相同"%(k+1),n1==n2)
print("总元素个数是否相同",len(newG)==len(G),len(G))

#### B,C,F 型-取前 4 层

In [ ]:
# 初始化
s,n = "F",4
mat = matrix(CoxeterMatrix([s,n]))
print("Coxeter 矩阵",mat,sep="\n")
G = GroupByOrderMatrix(mat)

# 计算结果
k = 4
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%k,relations, sep="\n")
print()
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements

# 打印结果
n1 = [len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]
n2 = [len(i) for i in GroupTreeOfMaxDepth(G,k+1)]
print("前%d层元素个数是否相同"%(k+1),n1==n2)
# print("群元素是否相同",len(newG)==len(G),len(G))

#### G 型-取前 6 层

In [ ]:
# 初始化
s,n = "G",2
mat = matrix(CoxeterMatrix([s,n]))
print("Coxeter 矩阵",mat,sep="\n")
G = GroupByOrderMatrix(mat)

# 计算结果
k = 6
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%k,relations, sep="\n")
print()
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements

# 打印结果
n1 = [len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]
n2 = [len(i) for i in GroupTreeOfMaxDepth(G,k+1)]
print("前%d层元素个数是否相同"%(k+1),n1==n2)
print("群元素是否相同",len(newG)==len(G),len(G))

### 测试七 Weyl 群扩张群的极小泛性质

#### A 型-筛减

In [ ]:
# 初始化
s,n = "A",4
gens = Thetas(s,n)
G = MatrixGroup(gens)

# 计算结果
k = 4
_,_,relations = UniversalPropertyOfGroup(G,k)
relations = {key:relations[key] for key in relations.keys() if len(set(list(key.replace("-",""))))<=2}
print("前%d层导出关系"%k,relations, sep="\n")
print()
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements

In [ ]:
# 打印结果
n1 = [len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]
n2 = [len(i) for i in GroupTreeOfMaxDepth(G,k+1)]
print("前%d层元素个数是否相同"%(k+1),n1==n2)
print("群元素是否相同",len(newG)==len(G),len(G))

#### ADE 型-取前 4 层 

In [ ]:
# 初始化
s,n = "A",3
gens = Thetas(s,n)
G = MatrixGroup(gens)

# 计算结果
k = 4
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%k,relations, sep="\n")
print()
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements

In [ ]:
# 打印结果
n1 = [len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]
n2 = [len(i) for i in GroupTreeOfMaxDepth(G,k+1)]
print("前%d层元素个数是否相同"%(k+1),n1==n2)
print("群元素是否相同",len(newG)==len(G),len(G))

#### BCF 型-取前 4 层

In [ ]:
# 初始化
s,n = "F",4
gens = Thetas(s,n)
G = MatrixGroup(gens)

# 计算结果
k = 4
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%k,relations, sep="\n")
print()
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements

In [ ]:
# 打印结果
n1 = [len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]
n2 = [len(i) for i in GroupTreeOfMaxDepth(G,k+1)]
print("前%d层元素个数是否相同"%(k+1),n1==n2)
print("群元素是否相同",len(newG)==len(G),len(G))

#### G 型-取前 6 层

In [ ]:
# 初始化
s,n = "G",2
gens = Thetas(s,n)
G = MatrixGroup(gens)

# 计算结果
k = 6
_,_,relations = UniversalPropertyOfGroup(G,k)
print("前%d层导出关系"%k,relations, sep="\n")
print()
# 定义商群
FG = FreeGroup(n)
elements = relations2elements(relations,FG.gens())
print("商集",elements,sep="\n")
newG = FG/elements

In [ ]:
# 打印结果
n1 = [len(i) for i in GroupTreeOfMaxDepth(newG,k+1)]
n2 = [len(i) for i in GroupTreeOfMaxDepth(G,k+1)]
print("前%d层元素个数是否相同"%(k+1),n1==n2)
print("群元素是否相同",len(newG)==len(G),len(G))